## 설치법: 

터미널에서

```
$ pip install numpy
$ pip install matplotlib
$ pip install pandas
$ pip install voila
```

## 접속법 1

터미널에서

```
$ jupyter notebook
```

jupyter notebook이 켜지면 아래 사진 모양 클릭

<img src="https://drive.google.com/uc?id=1sK-6PYqrUodwHHym8717utRZb3PJhj1Z" width="640" height="320">

## 접속법 2

터미널에서

```
$ voila Traffic.ipynb
```

---

**만약 그림에 한글이 안보이는 경우 "네이버"의 나눔바른고딕 TTF 설치후 진행**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.rcParams["font.family"] = 'NanumBarunGothic'

df = pd.read_csv("alldata_groupby.csv", encoding="euc-kr")
df["date"] = pd.to_datetime(df["date"])
df_group = df.copy()
df_group = df_group.groupby([pd.Grouper(key="date", freq="W-SUN"), "sregion", "eregion"]).sum()

#  (도착-출발) 차이 칼럼 추가
scale_e = ["e1", "e2", "e3", "e4", "e5", "e6"]
scale_s = ["s1", "s2", "s3", "s4", "s5", "s6"]
for i, (s, e) in enumerate(zip(scale_s, scale_e)):
    df_group[f"diff_{i+1}"] = df_group[s] - df_group[e]
df_group["diff_total"] = df_group["stotal"] - df_group["etotal"]

dates = pd.Series(df_group.index.get_level_values(0))
dates_tuple = list(zip(*(dates.dt.strftime("%m-%d").unique(), dates.unique())))
columns = df_group.columns[df_group.columns.str.match("diff")]
regions = list(df.sregion.unique())

In [2]:
def create_widget(dates, columns):
    style = {'description_width': 'initial'}
    date = widgets.Dropdown(options=dates, description='Date:', style=style)
    see = widgets.Dropdown(options=columns, description='See:', style=style)
    form = widgets.HBox([date, see], 
                        layout=widgets.Layout(display='inline-flex', flex_flow='row', border='solid 2px', justify_content='space-between'))
    interactive_dict = {"see": see, "date": date}
    return form, interactive_dict

def draw(date, see):
    global df_group
    df_p = df_group.unstack(-1).loc[date, see]
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 10))
    im = ax.pcolor(df_p, cmap="coolwarm")
    ax.set_xticks(np.arange(0.5, len(df_p.columns), 1))
    ax.set_xticklabels(df_p.columns)
    ax.set_xlabel("출구", fontsize=14)

    ax.set_yticks(np.arange(0.5, len(df_p.index), 1))
    ax.set_yticklabels(df_p.index)
    ax.set_ylabel("입구", fontsize=14)
    ax.set_title("Relation", fontsize=20)
    fig.colorbar(im)
    plt.show()
    
def show(dates, columns):
    form, interactive_dict = create_widget(dates, columns)
    out = widgets.interactive_output(draw, interactive_dict)        
    display(form, out)

1번시기: 1/1~1/23

2번시기: 1/28~2/23 

3번시기: 2/24~3/8

4번시기: 3/9~4/12

# 각 차종별 주간 통행량 유입/유출

각 차종별, 지역별로 출발지역 통행량에서 도착 지역 통행량을 차감

In [3]:
show(dates_tuple, columns)

Output()